# Profit Status Enrichment Notebook

This notebook enriches information sources with profit status data by using the Anthropic API to research each organization.

## Purpose
Automatically determine the profit status (non-profit, for-profit, government, mixed, or unknown) for sources in the Google Sheet and update the sheet with this information.

## Requirements
- **Credentials**: `credentials.json` file in the root directory (Google service account)
- **API Key**: Anthropic API key (set as environment variable `ANTHROPIC_API_KEY`)
- **Dependencies**: anthropic, google-api-python-client
- **Sheet Columns**: The sheet must have a `profit_status` column

## How it works
1. Connects to Google Sheets and loads source data
2. For each source without profit status:
   - Uses Claude API to research the organization
   - Determines profit status with confidence level
   - Updates the Google Sheet with the findings
3. Applies rate limiting to respect API usage limits

In [13]:
import json
import time
import os
from datetime import datetime
from anthropic import Anthropic
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from typing import Optional, Tuple, Dict

In [14]:
# Configuration
SERVICE_ACCOUNT_FILE = "/workspaces/info-sources/credentials.json"
SPREADSHEET_ID = "1NywRL9IBR69R0eSrOE9T6mVUbfJHwaALL0vp2K0TLbY"
SHEET_RANGE = "main!A:C"
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]

# API Configuration
DELAY_BETWEEN_REQUESTS = 1.0  # seconds

In [15]:
def research_profit_status(client: Anthropic, name: str, url: str) -> Optional[Dict[str, str]]:
    """
    Research an organization's profit status using the Anthropic API.
    
    Args:
        client: Anthropic API client
        name: Name of the organization
        url: URL of the organization
        
    Returns:
        Dictionary with profit_status, confidence, and brief_reasoning, or None if error
    """
    try:
        prompt = f"""Research the organization "{name}" ({url}) and determine its profit status.

Respond ONLY with a valid JSON object in this EXACT format with NO other text:
{{
  "profit_status": "one of: non-profit, for-profit, government, mixed, or unknown",
  "confidence": "high, medium, or low",
  "brief_reasoning": "one sentence explanation"
}}

DO NOT include any text outside the JSON. DO NOT use markdown code blocks."""

        message = client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=500,
            messages=[{"role": "user", "content": prompt}]
        )
        
        response_text = message.content[0].text.strip()
        response_text = response_text.replace("```json", "").replace("```", "").strip()
        result = json.loads(response_text)
        
        return result
        
    except Exception as e:
        print(f"  ⚠️ Error researching: {str(e)}")
        return None

In [16]:
def load_sheet_data():
    """
    Load data from Google Sheets.
    
    Returns:
        Tuple of (sheets_service, headers, data_rows)
    """
    print("🔗 Connecting to Google Sheets...")
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE,
        scopes=SCOPES
    )
    sheets_service = build("sheets", "v4", credentials=creds)
    print("✅ Connected to Google Sheets")
    
    print("📂 Loading data from Google Sheet...")
    sheet = sheets_service.spreadsheets()
    result = sheet.values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=SHEET_RANGE
    ).execute()
    
    values = result.get("values", [])
    
    if not values:
        raise ValueError("❌ No data found in sheet")
    
    # Parse headers and data
    headers = values[0]
    data_rows = []
    for i, row in enumerate(values[1:], start=1):
        # Pad row to match header length
        row_data = row + [''] * (len(headers) - len(row))
        row_dict = {headers[j]: row_data[j] for j in range(len(headers))}
        row_dict['_row_index'] = i + 1  # +1 for header row
        data_rows.append(row_dict)
    
    print(f"✅ Loaded {len(data_rows)} sources")
    return sheets_service, headers, data_rows


def update_sheet_row(sheets_service, row_index: int, headers: list, profit_status: str):
    """
    Update profit_status column in a specific row of the sheet.
    
    Args:
        sheets_service: Google Sheets service instance
        row_index: Row number in the sheet (1-indexed)
        headers: List of column headers
        profit_status: Profit status value to write
    """
    # Find column index
    status_col_idx = headers.index('profit_status') if 'profit_status' in headers else None
    
    if status_col_idx is None:
        raise ValueError("❌ Required column 'profit_status' not found in sheet")
    
    # Convert column index to letter (0->A, 1->B, etc.)
    def col_to_letter(col_idx):
        result = ""
        while col_idx >= 0:
            result = chr(65 + (col_idx % 26)) + result
            col_idx = col_idx // 26 - 1
        return result
    
    status_col = col_to_letter(status_col_idx)
    
    # Update profit status
    range_name = f"main!{status_col}{row_index}"
    body = {'values': [[profit_status]]}
    sheets_service.spreadsheets().values().update(
        spreadsheetId=SPREADSHEET_ID,
        range=range_name,
        valueInputOption='RAW',
        body=body
    ).execute()

In [17]:
def process_profit_status_enrichment():
    """
    Main workflow function that processes all sources and enriches them with profit status data.
    """
    try:
        # Get API key
        api_key = input("🔑 Enter your Anthropic API key: ").strip()
        
        # Initialize Anthropic client
        client = Anthropic(api_key=api_key)
        print("✅ Anthropic API client initialized")
        
        # Load sheet data
        sheets_service, headers, data_rows = load_sheet_data()
        
        # Verify required column exists
        if 'profit_status' not in headers:
            print("❌ Error: Required column 'profit_status' not found in sheet")
            print(f"📋 Available columns: {', '.join(headers)}")
            return
        
        # Count existing vs needed enrichment
        already_filled = sum(1 for row in data_rows if row.get('profit_status', '').strip())
        needs_enrichment = len(data_rows) - already_filled
        
        print(f"📊 Status: {already_filled} already have profit_status, {needs_enrichment} need research")
        print(f"🚀 Starting profit status enrichment...\n")
        
        # Process each row
        start_time = time.time()
        updated_count = 0
        skipped_count = 0
        failed_count = 0
        
        for idx, row in enumerate(data_rows):
            name = row.get('name', '').strip()
            url = row.get('url', '').strip()
            existing_status = row.get('profit_status', '').strip()
            row_index = row.get('_row_index')
            
            # Skip rows with missing data
            if not name or not url:
                print(f"⏭️  [{idx + 1}/{len(data_rows)}] Skipping row {row_index}: missing name or URL")
                continue
            
            # Skip rows that already have a status
            if existing_status:
                print(f"⏭️  [{idx + 1}/{len(data_rows)}] Skipping {name}: already has status '{existing_status}'")
                skipped_count += 1
                continue
            
            print(f"🔍 [{idx + 1}/{len(data_rows)}] Researching: {name}")
            print(f"   URL: {url}")
            
            # Research profit status
            result = research_profit_status(client, name, url)
            
            if result and result.get('profit_status'):
                # Update sheet with findings
                try:
                    update_sheet_row(sheets_service, row_index, headers, result['profit_status'])
                    updated_count += 1
                    print(f"   ✅ {result['profit_status']} (confidence: {result.get('confidence', 'unknown')})")
                    print(f"   💡 {result.get('brief_reasoning', 'No reasoning provided')}")
                    print(f"   📝 Updated sheet\n")
                except Exception as e:
                    failed_count += 1
                    print(f"   ❌ Error updating sheet: {str(e)}\n")
            else:
                failed_count += 1
                print(f"   ❌ Failed to determine profit status\n")
            
            # Apply rate limiting
            time.sleep(DELAY_BETWEEN_REQUESTS)
        
        # Print summary
        elapsed = time.time() - start_time
        print(f"\n{'='*60}")
        print(f"📊 Summary")
        print(f"{'='*60}")
        print(f"✅ Sources updated with profit status: {updated_count}")
        print(f"⏭️  Sources skipped (already had data): {skipped_count}")
        print(f"❌ Sources failed: {failed_count}")
        print(f"⏱️  Total time elapsed: {elapsed/60:.1f} minutes")
        print(f"{'='*60}\n")
        
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

In [18]:
# Execute the profit status enrichment process
process_profit_status_enrichment()

✅ Anthropic API client initialized
🔗 Connecting to Google Sheets...
✅ Connected to Google Sheets
📂 Loading data from Google Sheet...
✅ Loaded 359 sources
📊 Status: 353 already have profit_status, 6 need research
🚀 Starting profit status enrichment...

⏭️  [1/359] Skipping Insight Crime: already has status 'non-profit'
⏭️  [2/359] Skipping United States Institute of Peace: already has status 'government'
⏭️  [3/359] Skipping Pew Research: already has status 'non-profit'
⏭️  [4/359] Skipping ProPublica: already has status 'non-profit'
⏭️  [5/359] Skipping Associated Press: already has status 'non-profit'
⏭️  [6/359] Skipping Reuters: already has status 'for-profit'
⏭️  [7/359] Skipping Al Jazeera: already has status 'government'
⏭️  [8/359] Skipping Brookings Institute: already has status 'non-profit'
⏭️  [9/359] Skipping Business & Human Rights Resource Center: already has status 'https://www.business-humanrights.org/en/latest-news/?&language=en'
⏭️  [10/359] Skipping Armed Conflict Loc